# icw773r及iCGB21FR对比

In [ ]:
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model,load_json_model
import re
def get_metabolite_related_fluxes(model, metabolite_ids, fba_results):
    for metabolite_id in metabolite_ids:
        metabolite = model.metabolites.get_by_id(metabolite_id)
        print(f"Metabolite: {metabolite.name} ({metabolite_id})")
        
        for reaction in metabolite.reactions:
            coefficient = reaction.metabolites[metabolite]
            flux = fba_results[reaction.id]
            
            if coefficient != 0 and flux != 0:
                print(f"  Reaction: {reaction.id}")
                print(f"    Flux: {flux}")
                # print(f"    Coefficient: {coefficient}")
                
                # 构建反应方程式
                reactants = []
                products = []
                for met, coeff in reaction.metabolites.items():
                    if coeff < 0:  # 反应物
                        reactants.append(met.id)
                    elif coeff > 0:  # 生成物
                        products.append(met.id)
                
                # 打印反应方程式
                equation = " + ".join(reactants) + " → " + " + ".join(products)
                print(f"    Equation: {equation}")
                # print()
def add_reaction_method2(model, reaction_id, reaction_name, reaction_string, lower_bound=-1000.0, upper_bound=1000.0, subsystem=''):
    """
    使用反应字符串表达式向COBRA模型中添加一个新的反应
    
    参数:
    model: cobra.Model对象
    reaction_id: 反应ID (字符串)
    reaction_name: 反应名称 (字符串)
    reaction_string: 反应字符串表达式，例如 "A + B <=> C + D"
    lower_bound: 反应通量下限 (默认为-1000.0，表示可逆)
    upper_bound: 反应通量上限 (默认为1000.0)
    subsystem: 反应所属子系统 (字符串)
    
    返回:
    新添加的Reaction对象
    """
    # 检查模型对象是否有效
    if model is None:
        raise ValueError("模型对象不能为None")
    
    # 创建新的反应对象
    new_reaction = Reaction(reaction_id)
    new_reaction.name = reaction_name
    new_reaction.subsystem = subsystem
    
    # 手动处理反应字符串并添加代谢物
    try:
        # 解析反应字符串
        if "<=>" in reaction_string:
            parts = reaction_string.split("<=>")
            reversible = True
        elif "=>" in reaction_string:
            parts = reaction_string.split("=>")
            reversible = False
        elif "->" in reaction_string:
            parts = reaction_string.split("->")
            reversible = False
        else:
            raise ValueError(f"无法解析反应字符串: {reaction_string}")
        
        # 设置反应可逆性
        if reversible:
            new_reaction.lower_bound = lower_bound
        else:
            new_reaction.lower_bound = 0  # 不可逆反应
        
        new_reaction.upper_bound = upper_bound
        
        # 解析反应物和产物
        if len(parts) == 2:
            reactants_str = parts[0].strip()
            products_str = parts[1].strip()
            
            # 处理反应物
            metabolites_dict = {}
            
            # 解析反应物
            if reactants_str:
                for reactant in reactants_str.split("+"):
                    reactant = reactant.strip()
                    # 检查是否有系数
                    match = re.match(r'^\((\d+)\)\s+(.+)$', reactant)
                    if match:
                        coeff = -float(match.group(1))  # 反应物系数为负
                        met_id = match.group(2).strip()
                    else:
                        coeff = -1.0  # 默认系数为-1
                        met_id = reactant
                    
                    # 尝试从模型中获取代谢物，如果不存在则创建一个新的
                    try:
                        met = model.metabolites.get_by_id(met_id)
                    except KeyError:
                        print(f"警告: 代谢物 '{met_id}' 不存在于模型中，将创建一个新的代谢物")
                        # 从ID中提取基本ID和区室
                        if '_' in met_id and met_id.rsplit('_', 1)[1].startswith('c'):
                            base_id = met_id.rsplit('_', 1)[0]
                            compartment = met_id.rsplit('_', 1)[1]
                        else:
                            base_id = met_id
                            compartment = 'c'  # 默认为细胞质
                        
                        met = Metabolite(
                            id=met_id,
                            name=f"Metabolite {base_id}",
                            compartment=compartment
                        )
                        model.add_metabolites([met])
                    
                    metabolites_dict[met] = coeff
            
            # 解析产物
            if products_str:
                for product in products_str.split("+"):
                    product = product.strip()
                    # 检查是否有系数
                    match = re.match(r'^\((\d+)\)\s+(.+)$', product)
                    if match:
                        coeff = float(match.group(1))  # 产物系数为正
                        met_id = match.group(2).strip()
                    else:
                        coeff = 1.0  # 默认系数为1
                        met_id = product
                    
                    # 尝试从模型中获取代谢物，如果不存在则创建一个新的
                    try:
                        met = model.metabolites.get_by_id(met_id)
                    except KeyError:
                        print(f"警告: 代谢物 '{met_id}' 不存在于模型中，将创建一个新的代谢物")
                        # 从ID中提取基本ID和区室
                        if '_' in met_id and met_id.rsplit('_', 1)[1].startswith('c'):
                            base_id = met_id.rsplit('_', 1)[0]
                            compartment = met_id.rsplit('_', 1)[1]
                        else:
                            base_id = met_id
                            compartment = 'c'  # 默认为细胞质
                        
                        met = Metabolite(
                            id=met_id,
                            name=f"Metabolite {base_id}",
                            compartment=compartment
                        )
                        model.add_metabolites([met])
                    
                    metabolites_dict[met] = coeff
            
            # 添加代谢物到反应
            new_reaction.add_metabolites(metabolites_dict)
            
        else:
            raise ValueError(f"反应字符串格式不正确: {reaction_string}")
        
        # 将反应添加到模型中
        model.add_reactions([new_reaction])
        
        print(f"成功添加反应: {new_reaction.id}")
        print(f"反应方程式: {new_reaction.build_reaction_string()}")
        
        return new_reaction
    
    except Exception as e:
        print(f"添加反应时出错: {e}")
        raise
    
    return new_reaction
if __name__ == "__main__":
    # 加载模型
    file_path = "other model/iCW773R.xml"
    try:
        model = read_sbml_model(file_path)
        print(f"Model successfully loaded from {file_path}.")
    except Exception as e:
        print(f"Error loading model: {e}")
        model = None
    add_reaction_method2(model,'for12','for1','=> ppa_c')
    reaction_ids_1 = {'EX_glc_e_reverse'}
    for reaction_id in reaction_ids_1:
        if reaction_id in model.reactions:
            reaction = model.reactions.get_by_id(reaction_id)
            print(f"反应 {reaction_id} 原始上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
            reaction.upper_bound = 0
            reaction.lower_bound = 0# 设置下界
            print(f"反应 {reaction_id} 修改后上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
    reaction_ids_2 = {'for12'}
    for reaction_id in reaction_ids_2:
        if reaction_id in model.reactions:
            reaction = model.reactions.get_by_id(reaction_id)
            print(f"反应 {reaction_id} 原始上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
            reaction.lower_bound = 10# 设置下界
            reaction.upper_bound = reaction.lower_bound # 设置上界
            print(f"反应 {reaction_id} 修改后上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
        else:
            print(f"反应 {reaction_id} 不存在于模型中！")
    reaction_ids_2 = {}
    for reaction_id in reaction_ids_2:
        if reaction_id in model.reactions:
            reaction = model.reactions.get_by_id(reaction_id)
            print(f"反应 {reaction_id} 原始上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
            reaction.lower_bound = 0# 设置下界
            reaction.upper_bound = reaction.lower_bound # 设置上界
            print(f"反应 {reaction_id} 修改后上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
        else:
            print(f"反应 {reaction_id} 不存在于模型中！")
    
    biomass_rxn_id = "CG_biomass_cgl_ATCC13032"  # 生物量反应ID CG_biomass_cgl_ATCC13032
    biomass_rxn = model.reactions.get_by_id(biomass_rxn_id)
    model.objective = biomass_rxn
    fba_results = model.optimize()
    print(f"Objective Value (FBA): {fba_results.objective_value}")

    metabolite_ids = ['succ_c']
    get_metabolite_related_fluxes(model, metabolite_ids, fba_results)

In [ ]:
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model,load_json_model
import re
def get_metabolite_related_fluxes(model, metabolite_ids, fba_results):
    for metabolite_id in metabolite_ids:
        metabolite = model.metabolites.get_by_id(metabolite_id)
        print(f"Metabolite: {metabolite.name} ({metabolite_id})")
        
        for reaction in metabolite.reactions:
            coefficient = reaction.metabolites[metabolite]
            flux = fba_results[reaction.id]
            
            if coefficient != 0 and flux != 0:
                print(f"  Reaction: {reaction.id}")
                print(f"    Flux: {flux}")
                # print(f"    Coefficient: {coefficient}")
                
                # 构建反应方程式
                reactants = []
                products = []
                for met, coeff in reaction.metabolites.items():
                    if coeff < 0:  # 反应物
                        reactants.append(met.id)
                    elif coeff > 0:  # 生成物
                        products.append(met.id)
                
                # 打印反应方程式
                equation = " + ".join(reactants) + " → " + " + ".join(products)
                print(f"    Equation: {equation}")
                # print()
def add_reaction_method2(model, reaction_id, reaction_name, reaction_string, lower_bound=-1000.0, upper_bound=1000.0, subsystem=''):
    """
    使用反应字符串表达式向COBRA模型中添加一个新的反应
    
    参数:
    model: cobra.Model对象
    reaction_id: 反应ID (字符串)
    reaction_name: 反应名称 (字符串)
    reaction_string: 反应字符串表达式，例如 "A + B <=> C + D"
    lower_bound: 反应通量下限 (默认为-1000.0，表示可逆)
    upper_bound: 反应通量上限 (默认为1000.0)
    subsystem: 反应所属子系统 (字符串)
    
    返回:
    新添加的Reaction对象
    """
    # 检查模型对象是否有效
    if model is None:
        raise ValueError("模型对象不能为None")
    
    # 创建新的反应对象
    new_reaction = Reaction(reaction_id)
    new_reaction.name = reaction_name
    new_reaction.subsystem = subsystem
    
    # 手动处理反应字符串并添加代谢物
    try:
        # 解析反应字符串
        if "<=>" in reaction_string:
            parts = reaction_string.split("<=>")
            reversible = True
        elif "=>" in reaction_string:
            parts = reaction_string.split("=>")
            reversible = False
        elif "->" in reaction_string:
            parts = reaction_string.split("->")
            reversible = False
        else:
            raise ValueError(f"无法解析反应字符串: {reaction_string}")
        
        # 设置反应可逆性
        if reversible:
            new_reaction.lower_bound = lower_bound
        else:
            new_reaction.lower_bound = 0  # 不可逆反应
        
        new_reaction.upper_bound = upper_bound
        
        # 解析反应物和产物
        if len(parts) == 2:
            reactants_str = parts[0].strip()
            products_str = parts[1].strip()
            
            # 处理反应物
            metabolites_dict = {}
            
            # 解析反应物
            if reactants_str:
                for reactant in reactants_str.split("+"):
                    reactant = reactant.strip()
                    # 检查是否有系数
                    match = re.match(r'^\((\d+)\)\s+(.+)$', reactant)
                    if match:
                        coeff = -float(match.group(1))  # 反应物系数为负
                        met_id = match.group(2).strip()
                    else:
                        coeff = -1.0  # 默认系数为-1
                        met_id = reactant
                    
                    # 尝试从模型中获取代谢物，如果不存在则创建一个新的
                    try:
                        met = model.metabolites.get_by_id(met_id)
                    except KeyError:
                        print(f"警告: 代谢物 '{met_id}' 不存在于模型中，将创建一个新的代谢物")
                        # 从ID中提取基本ID和区室
                        if '_' in met_id and met_id.rsplit('_', 1)[1].startswith('c'):
                            base_id = met_id.rsplit('_', 1)[0]
                            compartment = met_id.rsplit('_', 1)[1]
                        else:
                            base_id = met_id
                            compartment = 'c'  # 默认为细胞质
                        
                        met = Metabolite(
                            id=met_id,
                            name=f"Metabolite {base_id}",
                            compartment=compartment
                        )
                        model.add_metabolites([met])
                    
                    metabolites_dict[met] = coeff
            
            # 解析产物
            if products_str:
                for product in products_str.split("+"):
                    product = product.strip()
                    # 检查是否有系数
                    match = re.match(r'^\((\d+)\)\s+(.+)$', product)
                    if match:
                        coeff = float(match.group(1))  # 产物系数为正
                        met_id = match.group(2).strip()
                    else:
                        coeff = 1.0  # 默认系数为1
                        met_id = product
                    
                    # 尝试从模型中获取代谢物，如果不存在则创建一个新的
                    try:
                        met = model.metabolites.get_by_id(met_id)
                    except KeyError:
                        print(f"警告: 代谢物 '{met_id}' 不存在于模型中，将创建一个新的代谢物")
                        # 从ID中提取基本ID和区室
                        if '_' in met_id and met_id.rsplit('_', 1)[1].startswith('c'):
                            base_id = met_id.rsplit('_', 1)[0]
                            compartment = met_id.rsplit('_', 1)[1]
                        else:
                            base_id = met_id
                            compartment = 'c'  # 默认为细胞质
                        
                        met = Metabolite(
                            id=met_id,
                            name=f"Metabolite {base_id}",
                            compartment=compartment
                        )
                        model.add_metabolites([met])
                    
                    metabolites_dict[met] = coeff
            
            # 添加代谢物到反应
            new_reaction.add_metabolites(metabolites_dict)
            
        else:
            raise ValueError(f"反应字符串格式不正确: {reaction_string}")
        
        # 将反应添加到模型中
        model.add_reactions([new_reaction])
        
        print(f"成功添加反应: {new_reaction.id}")
        print(f"反应方程式: {new_reaction.build_reaction_string()}")
        
        return new_reaction
    
    except Exception as e:
        print(f"添加反应时出错: {e}")
        raise
    
    return new_reaction
if __name__ == "__main__":
    # 加载模型
    file_path = "other model/iCGB21FR.xml"
    try:
        model = read_sbml_model(file_path)
        print(f"Model successfully loaded from {file_path}.")
    except Exception as e:
        print(f"Error loading model: {e}")
        model = None
    add_reaction_method2(model,'xx1','GLYK','<=> trp__L_c')
    # add_reaction_method2(model,'XYLK','XYLK','atp_c + xylu__D_c <=> adp_c + h_c + xu5p__D_c')
    # add_reaction_method2(model,'for11','for','glu__D_c + atp_c + (2) nadph_c + (2) h_c  => pi_c + h2o_c + adp_c + (2) nadp_c + pro__L_c')
    # add_reaction_method2(model,'for12','for1','glu__D_c + atp_c + nadph_c + (2) h_c + nadh_c => pi_c + h2o_c + adp_c + nadp_c + nad_c + pro__L_c')
    reaction_ids_1 = {'EX_nh4_e','EX_urea_e'}
    for reaction_id in reaction_ids_1:
        if reaction_id in model.reactions:
            reaction = model.reactions.get_by_id(reaction_id)
            print(f"反应 {reaction_id} 原始上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
            reaction.upper_bound = 0
            reaction.lower_bound = 0# 设置下界
            print(f"反应 {reaction_id} 修改后上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")

    biomass_rxn_id = "Growth"  # 生物量反应ID CG_biomass_cgl_ATCC13032
    biomass_rxn = model.reactions.get_by_id(biomass_rxn_id)
    model.objective = biomass_rxn
    fba_results = model.optimize()
    print(f"Objective Value (FBA): {fba_results.objective_value}")
    
    metabolite_ids = ['trp__L_c']
    get_metabolite_related_fluxes(model, metabolite_ids, fba_results)

In [1]:
import cobra
import pandas as pd
import os

def sbml_to_excel(model_path, output_excel_path):
    """
    将SBML格式的代谢模型转换为Excel表格

    参数:
    model_path (str): SBML模型文件路径
    output_excel_path (str): 输出Excel文件路径
    """
    # 加载模型
    model = cobra.io.read_sbml_model(model_path)

    # 创建一个Excel写入器
    writer = pd.ExcelWriter(output_excel_path, engine='openpyxl')

    # 1. 反应表
    reaction_data = []
    for rxn in model.reactions:
        gene_association = rxn.gene_reaction_rule if hasattr(rxn, 'gene_reaction_rule') else ""
        reaction_data.append({
            'ID': rxn.id,
            'Name': rxn.name,
            'Equation': rxn.build_reaction_string(),
            'Lower Bound': rxn.lower_bound,
            'Upper Bound': rxn.upper_bound,
            'Gene Association': gene_association,
            'Subsystem': rxn.subsystem if hasattr(rxn, 'subsystem') else "",
            'Genes': ", ".join([g.id for g in rxn.genes])
        })

    reactions_df = pd.DataFrame(reaction_data)
    reactions_df.to_excel(writer, sheet_name='Reactions', index=False)

    # 2. 代谢物表
    metabolite_data = []
    for met in model.metabolites:
        metabolite_data.append({
            'ID': met.id,
            'Name': met.name,
            'Formula': met.formula if hasattr(met, 'formula') else "",
            'Charge': met.charge if hasattr(met, 'charge') else "",
            'Compartment': met.compartment if hasattr(met, 'compartment') else "",
            'In Reactions': ", ".join([r.id for r in met.reactions])
        })

    metabolites_df = pd.DataFrame(metabolite_data)
    metabolites_df.to_excel(writer, sheet_name='Metabolites', index=False)

    # 3. 基因表
    gene_data = []
    for gene in model.genes:
        gene_data.append({
            'ID': gene.id,
            'Name': gene.name if hasattr(gene, 'name') else "",
            'In Reactions': ", ".join([r.id for r in gene.reactions])
        })

    genes_df = pd.DataFrame(gene_data)
    genes_df.to_excel(writer, sheet_name='Genes', index=False)

    # 4. 模型信息表
    model_info = {
        'Property': ['Model ID', 'Number of Reactions', 'Number of Metabolites', 'Number of Genes'],
        'Value': [model.id, len(model.reactions), len(model.metabolites), len(model.genes)]
    }
    model_info_df = pd.DataFrame(model_info)
    model_info_df.to_excel(writer, sheet_name='Model Info', index=False)

    # 保存Excel文件
    writer.close()

    print(f"模型已成功转换为Excel文件: {output_excel_path}")

# 示例使用
if __name__ == "__main__":
    model_path = "other model/iCGB21FR.xml"  # 替换为您的SBML模型路径
    output_excel_path = "other model/iCGB21_model.xlsx"  # 输出Excel文件路径

    sbml_to_excel(model_path, output_excel_path)

Set parameter Username
Set parameter LicenseID to value 2723056
Academic license - for non-commercial use only - expires 2026-10-16


No objective coefficients in model. Unclear what should be optimized


模型已成功转换为Excel文件: other model/iCGB21_model.xlsx


# 140067途径验证

In [5]:
import pandas as pd
from cobra.io import read_sbml_model
from cobra import Reaction
from cobra.flux_analysis import pfba  # 导入pfba函数
import os
from datetime import datetime

def get_metabolite_related_fluxes_simple(model, metabolite_ids, fba_results, reaction_data=None):
    """简化版本：直接使用cobra模型中的信息"""
    for metabolite_id in metabolite_ids:
        try:
            metabolite = model.metabolites.get_by_id(metabolite_id)
            print(f"Metabolite: {metabolite.name} ({metabolite_id})")

            for reaction in metabolite.reactions:
                coefficient = reaction.metabolites[metabolite]
                flux = fba_results[reaction.id]

                if coefficient != 0 and flux != 0:
                    print(f"  Reaction: {reaction.id}")
                    print(f"    Flux: {flux}")
                    print(f"    Definition: {reaction.name}")
                    # 新增：从reaction_data中获取注释
                    if reaction_data and reaction.id in reaction_data:
                        print(f"    注释: {reaction_data[reaction.id].get('Definition', 'N/A')}")
                    print(f"    Equation: {reaction.reaction}")
                    print()
        except KeyError:
            print(f"Metabolite {metabolite_id} not found in model")

def get_specific_reaction_fluxes(model, reaction_ids, fba_results, reaction_data=None):
    """
    查看特定反应的通量信息

    Parameters:
    -----------
    model : cobra.Model
        代谢模型
    reaction_ids : list
        要查看的反应ID列表
    fba_results : pandas.Series
        FBA/pFBA求解结果的通量
    reaction_data : dict, optional
        从Excel读取的反应注释数据，格式: {reaction_id: {'Definition': ..., 'Equation': ...}}
    """
    print("=== 特定反应通量分析 ===")
    for reaction_id in reaction_ids:
        try:
            reaction = model.reactions.get_by_id(reaction_id)
            flux = fba_results[reaction_id]

            print(f"\nReaction ID: {reaction_id}")

            # 新增：从reaction_data中获取注释（Definition）
            if reaction_data and reaction_id in reaction_data:
                definition = reaction_data[reaction_id].get('Definition', 'N/A')
                print(f"  📝 注释(Definition): {definition}")
            else:
                print(f"  📝 注释(Definition): (未在Excel中找到)")

            print(f"  Flux: {flux}")
            print(f"  Equation: {reaction.reaction}")

            # 新增：从reaction_data中获取Equation（如果与模型不同可以对比）
            if reaction_data and reaction_id in reaction_data:
                excel_equation = reaction_data[reaction_id].get('Equation', 'N/A')

            # 如果通量为0，说明该反应没有活性
            if abs(flux) < 1e-6:
                print(f"  ⚠️  通量接近0，该反应可能未激活")
            elif flux > 0:
                print(f"  ✅ 正向反应活跃 (flux = {flux:.6f})")
            else:
                print(f"  🔄 反向反应活跃 (flux = {flux:.6f})")

        except KeyError:
            print(f"\n❌ Reaction {reaction_id} not found in model")

def search_reactions_by_keyword(model, keyword, fba_results=None, reaction_data=None):
    """根据关键词搜索反应"""
    print(f"\n=== 搜索包含关键词 '{keyword}' 的反应 ===")
    found_reactions = []

    for reaction in model.reactions:
        # 在反应ID、名称和方程式中搜索关键词
        # 新增：也在Definition中搜索
        definition = ""
        if reaction_data and reaction.id in reaction_data:
            definition = reaction_data[reaction.id].get('Definition', '')

        if (keyword.lower() in reaction.id.lower() or
            keyword.lower() in reaction.name.lower() or
            keyword.lower() in reaction.reaction.lower() or
            keyword.lower() in definition.lower()):
            found_reactions.append(reaction.id)

            print(f"\nReaction ID: {reaction.id}")
            print(f"  Name: {reaction.name}")
            if definition:
                print(f"  📝 注释(Definition): {definition}")
            if fba_results is not None:
                flux = fba_results[reaction.id]
                print(f"  Flux: {flux:.6f}")
            print(f"  Equation: {reaction.reaction}")

    print(f"\n找到 {len(found_reactions)} 个相关反应")
    return found_reactions

def get_all_nonzero_fluxes(fba_results, threshold=1e-6):
    """获取所有非零通量的反应"""
    print(f"\n=== 所有非零通量反应 (阈值: {threshold}) ===")
    nonzero_fluxes = []

    for reaction_id, flux in fba_results.items():
        if abs(flux) > threshold:
            nonzero_fluxes.append((reaction_id, flux))

    # 按通量绝对值排序
    nonzero_fluxes.sort(key=lambda x: abs(x[1]), reverse=True)

    print(f"共找到 {len(nonzero_fluxes)} 个活跃反应")
    for i, (reaction_id, flux) in enumerate(nonzero_fluxes[:20]):  # 只显示前20个
        print(f"{i+1:2d}. {reaction_id}: {flux:.6f}")

    if len(nonzero_fluxes) > 20:
        print(f"... 还有 {len(nonzero_fluxes) - 20} 个反应")

    return [r[0] for r in nonzero_fluxes]

def add_demand_reaction(model, metabolite_id):
    # 创建一个新的需求反应
    demand_reaction_id = f"DM_{metabolite_id}"
    demand_reaction = Reaction(demand_reaction_id)
    demand_reaction.name = f"Demand reaction for {metabolite_id}"
    demand_reaction.subsystem = 'Demand'

    # 获取代谢物对象
    metabolite = model.metabolites.get_by_id(metabolite_id)

    # 设置需求反应的化学计量
    demand_reaction.add_metabolites({metabolite: -1})

    # 设置需求反应的上下限
    demand_reaction.lower_bound = 0
    demand_reaction.upper_bound = 1000

    # 将需求反应添加到模型中
    model.add_reactions([demand_reaction])

    return demand_reaction

def count_active_reactions(fba_results, threshold=1e-6):
    """
    统计活跃反应数量

    Parameters:
    -----------
    fba_results : pandas.Series or dict
        FBA/pFBA求解结果的通量
    threshold : float
        判断活跃的阈值，默认1e-6

    Returns:
    --------
    tuple : (活跃反应数, 总反应数, 活跃反应列表)
    """
    active_reactions = []

    for reaction_id, flux in fba_results.items():
        if abs(flux) > threshold:
            active_reactions.append((reaction_id, flux))

    total_reactions = len(fba_results)
    active_count = len(active_reactions)

    return active_count, total_reactions, active_reactions


# ==================== 新增：辅因子成本分析函数 ====================

def analyze_cofactor_costs(model, pfba_results, target_product_id, cofactors=None,
                          reaction_data=None, top_n=3):
    """
    分析辅因子成本（ATP、NADH、NADPH等）

    Parameters:
    -----------
    model : cobra.Model
        代谢模型
    pfba_results : cobra.Solution
        pFBA求解结果
    target_product_id : str
        目标产物反应ID，用于计算单位成本
    cofactors : dict, optional
        辅因子定义，格式: {'辅因子名称': ['代谢物ID列表']}
        默认包含ATP、NADH、NADPH
    reaction_data : dict, optional
        从Excel读取的反应注释数据
    top_n : int
        显示前n个主要反应

    Returns:
    --------
    dict : 辅因子分析结果
    """
    # 默认辅因子定义
    if cofactors is None:
        cofactors = {
            'ATP': ['cpd00002_c'],      # ATP
            'NADH': ['cpd00004_c'],     # NADH
            'NADPH': ['cpd00006_c']     # NADPH
        }

    print(f"\n💰 辅因子成本分析")
    print("=" * 70)

    # 获取目标产物通量
    target_flux = pfba_results.fluxes.get(target_product_id, 0)
    print(f"🎯 目标产物: {target_product_id}")
    print(f"   目标通量: {target_flux:.4f} mmol/gDCW/h")

    results = {}

    for cofactor_name, metabolite_ids in cofactors.items():
        total_production = 0
        total_consumption = 0
        production_rxns = []
        consumption_rxns = []

        for met_id in metabolite_ids:
            try:
                metabolite = model.metabolites.get_by_id(met_id)

                for reaction in metabolite.reactions:
                    coefficient = reaction.metabolites[metabolite]
                    flux = pfba_results.fluxes[reaction.id]
                    contribution = coefficient * flux

                    if abs(contribution) > 1e-6:
                        # 获取反应注释
                        definition = ""
                        if reaction_data and reaction.id in reaction_data:
                            definition = reaction_data[reaction.id].get('Definition', '')

                        if contribution > 0:
                            total_production += contribution
                            production_rxns.append({
                                'id': reaction.id,
                                'name': reaction.name,
                                'definition': definition,
                                'flux': flux,
                                'contribution': contribution
                            })
                        else:
                            total_consumption += abs(contribution)
                            consumption_rxns.append({
                                'id': reaction.id,
                                'name': reaction.name,
                                'definition': definition,
                                'flux': flux,
                                'contribution': abs(contribution)
                            })

            except KeyError:
                print(f"⚠️  代谢物 {met_id} 未找到")

        # 排序
        production_rxns.sort(key=lambda x: x['contribution'], reverse=True)
        consumption_rxns.sort(key=lambda x: x['contribution'], reverse=True)

        # 计算单位成本
        unit_cost = 0
        if abs(target_flux) > 1e-6:
            unit_cost = total_consumption / abs(target_flux)

        results[cofactor_name] = {
            'production': total_production,
            'consumption': total_consumption,
            'net': total_production - total_consumption,
            'unit_cost': unit_cost,
            'production_rxns': production_rxns,
            'consumption_rxns': consumption_rxns
        }

    # 输出结果
    for cofactor, data in results.items():
        print(f"\n💎 {cofactor}")
        print(f"   生产量: {data['production']:>10.4f} mmol/gDCW/h")
        print(f"   消耗量: {data['consumption']:>10.4f} mmol/gDCW/h")

        # 净通量检查
        net_status = '✅ 平衡' if abs(data['net']) < 1e-4 else '⚠️ 不平衡'
        print(f"   净通量: {data['net']:>10.4f} {net_status}")

        # 单位成本
        if abs(target_flux) > 1e-6:
            print(f"   📊 单位成本: {data['unit_cost']:>6.3f} mol {cofactor}/mol 产物")

        # 显示主要生产反应
        if data['production_rxns']:
            print(f"   🔺 主要生产反应 (Top {min(top_n, len(data['production_rxns']))}):")
            for rxn in data['production_rxns'][:top_n]:
                definition = rxn['definition'][:30] + "..." if len(rxn['definition']) > 30 else rxn['definition']
                print(f"      {rxn['id']:25s} +{rxn['contribution']:8.4f}  {definition}")

        # 显示主要消耗反应
        if data['consumption_rxns']:
            print(f"   🔻 主要消耗反应 (Top {min(top_n, len(data['consumption_rxns']))}):")
            for rxn in data['consumption_rxns'][:top_n]:
                definition = rxn['definition'][:30] + "..." if len(rxn['definition']) > 30 else rxn['definition']
                print(f"      {rxn['id']:25s} -{rxn['contribution']:8.4f}  {definition}")

    # 打印汇总表
    print(f"\n📋 辅因子成本汇总表")
    print("-" * 70)
    print(f"{'辅因子':<10} {'生产量':>12} {'消耗量':>12} {'净通量':>12} {'单位成本':>15}")
    print("-" * 70)
    for cofactor, data in results.items():
        print(f"{cofactor:<10} {data['production']:>12.4f} {data['consumption']:>12.4f} "
              f"{data['net']:>12.4f} {data['unit_cost']:>12.4f} mol/mol")
    print("-" * 70)

    return results


def export_cofactor_analysis_to_csv(results, output_file="cofactor_analysis.csv"):
    """
    将辅因子分析结果导出为CSV文件

    Parameters:
    -----------
    results : dict
        analyze_cofactor_costs的返回结果
    output_file : str
        输出文件路径

    Returns:
    --------
    str : 输出文件路径
    """
    print(f"\n📁 导出辅因子分析结果到CSV文件...")

    # 汇总数据
    summary_data = []
    for cofactor, data in results.items():
        summary_data.append({
            'Cofactor': cofactor,
            'Production': data['production'],
            'Consumption': data['consumption'],
            'Net_Flux': data['net'],
            'Unit_Cost_mol_per_mol': data['unit_cost']
        })

    summary_df = pd.DataFrame(summary_data)
    summary_file = output_file.replace('.csv', '_summary.csv')
    summary_df.to_csv(summary_file, index=False, encoding='utf-8-sig')
    print(f"✅ 汇总已导出到: {summary_file}")

    # 详细反应数据
    detail_data = []
    for cofactor, data in results.items():
        for rxn in data['production_rxns']:
            detail_data.append({
                'Cofactor': cofactor,
                'Type': 'Production',
                'Reaction_ID': rxn['id'],
                'Reaction_Name': rxn['name'],
                'Definition': rxn['definition'],
                'Flux': rxn['flux'],
                'Contribution': rxn['contribution']
            })
        for rxn in data['consumption_rxns']:
            detail_data.append({
                'Cofactor': cofactor,
                'Type': 'Consumption',
                'Reaction_ID': rxn['id'],
                'Reaction_Name': rxn['name'],
                'Definition': rxn['definition'],
                'Flux': rxn['flux'],
                'Contribution': -rxn['contribution']  # 负值表示消耗
            })

    detail_df = pd.DataFrame(detail_data)
    detail_df = detail_df.sort_values(['Cofactor', 'Type', 'Contribution'],
                                       ascending=[True, True, False])
    detail_file = output_file.replace('.csv', '_details.csv')
    detail_df.to_csv(detail_file, index=False, encoding='utf-8-sig')
    print(f"✅ 详细反应已导出到: {detail_file}")

    return {'summary': summary_file, 'details': detail_file}


# ==================== 其他原有函数 ====================

def export_flux_to_csv(model, pfba_results, output_file="flux_results.csv",
                       threshold=1e-6, only_active=False, reaction_data=None):
    """
    将通量结果导出为CSV文件
    """
    print(f"\n📁 导出通量结果到CSV文件...")

    # 收集数据
    data = []
    for reaction in model.reactions:
        flux = pfba_results.fluxes[reaction.id]
        abs_flux = abs(flux)
        is_active = abs_flux > threshold

        # 如果只导出活跃反应，跳过非活跃的
        if only_active and not is_active:
            continue

        # 获取Definition注释
        definition = ""
        excel_equation = ""
        if reaction_data and reaction.id in reaction_data:
            definition = reaction_data[reaction.id].get('Definition', '')
            excel_equation = reaction_data[reaction.id].get('Equation', '')

        data.append({
            'Reaction_ID': reaction.id,
            'Reaction_Name': reaction.name,
            'Definition': definition,
            'Flux': flux,
            'Abs_Flux': abs_flux,
            'Is_Active': is_active,
            'Lower_Bound': reaction.lower_bound,
            'Upper_Bound': reaction.upper_bound,
            'Subsystem': reaction.subsystem if reaction.subsystem else '',
            'Equation_Model': reaction.reaction,
            'Equation_Excel': excel_equation,
            'Gene_Reaction_Rule': reaction.gene_reaction_rule if reaction.gene_reaction_rule else ''
        })

    # 创建DataFrame
    df = pd.DataFrame(data)

    # 按绝对通量排序（降序）
    df = df.sort_values('Abs_Flux', ascending=False)

    # 导出CSV
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

    # 统计信息
    active_count = df['Is_Active'].sum()
    total_count = len(df) if not only_active else active_count

    print(f"✅ 已导出到: {output_file}")
    print(f"   总反应数: {len(model.reactions)}")
    print(f"   导出反应数: {len(df)}")
    print(f"   活跃反应数: {active_count}")

    return output_file

def export_flux_summary_to_csv(model, pfba_results, output_prefix="flux_analysis",
                               threshold=1e-6, reaction_data=None):
    """
    导出完整的通量分析结果，包括多个CSV文件
    """
    output_files = {}

    # 1. 导出所有反应通量
    all_flux_file = f"{output_prefix}_all_fluxes.csv"
    export_flux_to_csv(model, pfba_results, all_flux_file, threshold,
                       only_active=False, reaction_data=reaction_data)
    output_files['all_fluxes'] = all_flux_file

    # 2. 导出活跃反应通量
    active_flux_file = f"{output_prefix}_active_fluxes.csv"
    export_flux_to_csv(model, pfba_results, active_flux_file, threshold,
                       only_active=True, reaction_data=reaction_data)
    output_files['active_fluxes'] = active_flux_file

    # 3. 导出统计摘要
    summary_file = f"{output_prefix}_summary.csv"

    active_count, total_count, _ = count_active_reactions(pfba_results.fluxes, threshold)
    fluxes = pfba_results.fluxes
    active_fluxes = [abs(f) for f in fluxes if abs(f) > threshold]

    summary_data = {
        'Metric': [
            'Total_Reactions',
            'Active_Reactions',
            'Inactive_Reactions',
            'Active_Ratio_%',
            'Objective_Value',
            'Solution_Status',
            'Max_Flux',
            'Min_Active_Flux',
            'Mean_Active_Flux',
            'Analysis_Threshold'
        ],
        'Value': [
            total_count,
            active_count,
            total_count - active_count,
            f"{active_count/total_count*100:.2f}",
            f"{pfba_results.objective_value:.6f}",
            pfba_results.status,
            f"{max(active_fluxes):.6f}" if active_fluxes else "N/A",
            f"{min(active_fluxes):.6f}" if active_fluxes else "N/A",
            f"{sum(active_fluxes)/len(active_fluxes):.6f}" if active_fluxes else "N/A",
            f"{threshold}"
        ]
    }

    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(summary_file, index=False, encoding='utf-8-sig')
    output_files['summary'] = summary_file

    print(f"\n📊 统计摘要已导出到: {summary_file}")

    return output_files

def print_flux_summary(model, pfba_results, threshold=1e-6, reaction_data=None):
    """
    打印通量分析总结报告
    """
    print("\n" + "=" * 70)
    print("📊 通量分析总结报告")
    print("=" * 70)

    # 统计活跃反应数
    active_count, total_count, active_reactions = count_active_reactions(
        pfba_results.fluxes, threshold
    )

    print(f"\n🔢 反应统计:")
    print(f"   活跃反应数: {active_count}/{total_count}")
    print(f"   活跃比例: {active_count/total_count*100:.1f}%")
    print(f"   非活跃反应数: {total_count - active_count}")

    # 解状态
    print(f"\n⚙️  求解状态:")
    print(f"   状态: {pfba_results.status}")
    print(f"   目标函数值: {pfba_results.objective_value:.6f}")

    # 通量统计
    fluxes = pfba_results.fluxes
    active_fluxes = [abs(f) for f in fluxes if abs(f) > threshold]

    if active_fluxes:
        print(f"\n📈 活跃通量统计:")
        print(f"   最大通量: {max(active_fluxes):.4f}")
        print(f"   最小通量: {min(active_fluxes):.6f}")
        print(f"   平均通量: {sum(active_fluxes)/len(active_fluxes):.4f}")

    # 按通量大小排序，显示top反应
    sorted_active = sorted(active_reactions, key=lambda x: abs(x[1]), reverse=True)

    print(f"\n🔝 Top 10 活跃反应:")
    print(f"   {'反应ID':<25} {'通量':>12} {'注释':<30}")
    print("   " + "-" * 70)
    for rxn_id, flux in sorted_active[:10]:
        # 获取注释
        definition = ""
        if reaction_data and rxn_id in reaction_data:
            definition = reaction_data[rxn_id].get('Definition', '')
            # 截断过长的注释
            if len(definition) > 28:
                definition = definition[:25] + "..."
        print(f"   {rxn_id:<25} {flux:>12.4f} {definition:<30}")

    print("\n" + "=" * 70)

    return active_count, total_count


if __name__ == "__main__":
    # 加载模型
    file_path = "iCZ871_CGXII.xml"
    try:
        model = read_sbml_model(file_path)
        print(f"Model successfully loaded from {file_path}.")
    except Exception as e:
        print(f"Error loading model: {e}")
        model = None
        exit()

    # 加载 Excel 文件中的反应定义和方程式
    excel_file_path = "14067gem20_MM.xlsx"
    reaction_data = {}  # 初始化为空字典
    try:
        df = pd.read_excel(excel_file_path, sheet_name="reactions")
        # 创建 ID 到 Definition 和 Equation 的映射
        reaction_data = {
            row["ID"]: {"Definition": row["Definition"], "Equation": row["Equation"]}
            for index, row in df.iterrows()
        }
        print(f"Reaction data successfully loaded from {excel_file_path}.")
        print(f"   共加载 {len(reaction_data)} 个反应注释")
    except Exception as e:
        print(f"Error loading reaction data: {e}")
        reaction_data = {}

    # 设置目标函数并运行 pFBA
    biomass_rxn_id = "EX_biomass_c"
    # add_reaction_method2(model,'rxn04784_c','rxn04784_c','cpd00002_c + cpd00047_c + cpd02851_c => cpd00008_c + cpd00009_c + cpd02884_c')
    reaction_ids_1 = {}
    for reaction_id in reaction_ids_1:
        if reaction_id in model.reactions:
            reaction = model.reactions.get_by_id(reaction_id)
            print(f"反应 {reaction_id} 原始上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
            reaction.lower_bound = -10
            reaction.upper_bound = 0
            print(f"反应 {reaction_id} 修改后上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
        else:
            print(f"反应 {reaction_id} 不存在于模型中！")
    reaction_ids_1 = {'EX_cpd00027_e'}
    for reaction_id in reaction_ids_1:
        if reaction_id in model.reactions:
            reaction = model.reactions.get_by_id(reaction_id)
            print(f"反应 {reaction_id} 原始上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
            reaction.lower_bound = -10
            reaction.upper_bound = 0
            print(f"反应 {reaction_id} 修改后上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
        else:
            print(f"反应 {reaction_id} 不存在于模型中！")
    try:
        biomass_rxn = model.reactions.get_by_id(biomass_rxn_id)
        model.objective = biomass_rxn
        pfba_results = pfba(model)

        # 打印通量总结（传入reaction_data以显示注释）
        print_flux_summary(model, pfba_results, reaction_data=reaction_data)

        # 分析辅因子成本（ATP、NADH、NADPH）
        cofactor_results = analyze_cofactor_costs(
            model=model,
            pfba_results=pfba_results,
            target_product_id=biomass_rxn_id,  # 目标产物ID
            cofactors={
                'ATP': ['cpd00002_c'],      # ATP
                'NADH': ['cpd00004_c'],     # NADH
                'NADPH': ['cpd00006_c']     # NADPH
            },
            reaction_data=reaction_data,
            top_n=5
        )
        specific_reactions = ['EX_biomass_c']
        get_specific_reaction_fluxes(model, specific_reactions, pfba_results.fluxes,
                                     reaction_data=reaction_data)
        metabolite_ids = []
        get_metabolite_related_fluxes_simple(model, metabolite_ids, pfba_results.fluxes,
                                             reaction_data=reaction_data)
    except Exception as e:
        print(f"Error during pFBA analysis: {e}")
        import traceback
        traceback.print_exc()

No objective coefficients in model. Unclear what should be optimized


Model successfully loaded from iCZ871_CGXII.xml.
Reaction data successfully loaded from 14067gem20_MM.xlsx.
   共加载 1753 个反应注释
反应 EX_cpd00027_e 原始上下限: -5.1 至 1000.0
反应 EX_cpd00027_e 修改后上下限: -10 至 0

📊 通量分析总结报告

🔢 反应统计:
   活跃反应数: 320/1753
   活跃比例: 18.3%
   非活跃反应数: 1433

⚙️  求解状态:
   状态: optimal
   目标函数值: 566.699784

📈 活跃通量统计:
   最大通量: 55.8990
   最小通量: 0.000079
   平均通量: 1.7709

🔝 Top 10 活跃反应:
   反应ID                                通量 注释                            
   ----------------------------------------------------------------------
   rxn08173_c                     55.8990 (1) ADP_c + (1) Phosphate...  
   EX_cpd00001_e                  40.2566 (1) H2O_e <=>                 
   rxn05319_c                     40.2566 (1) H2O_c => (1) H2O_e        
   rxnbccaa3_c                    38.7435 (1) Menaquinol 8_c + (6) ...  
   rxn08971_c                     29.3897 (1) NADH_c + (1) H+_c + (...  
   EX_cpd00011_e                  20.5206 (1) CO2_e =>                  
   rxn05467_c         

In [ ]:
# 氧气摄取率对菌毛蛋白产量的敏感性分析
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model
from cobra.flux_analysis import pfba

# 加载模型
model = read_sbml_model("iCZ871_CGXII.xml")
model.objective = "EX_CLPbio_c"
reaction_ids_1 = {'EX_cpd00027_e'}
for reaction_id in reaction_ids_1:
        if reaction_id in model.reactions:
            reaction = model.reactions.get_by_id(reaction_id)
            print(f"反应 {reaction_id} 原始上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
            reaction.lower_bound = -10
            reaction.upper_bound = 0
            print(f"反应 {reaction_id} 修改后上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
        else:
            print(f"反应 {reaction_id} 不存在于模型中！")
# 氧气范围 (0到-20，共21个点)
o2_range = np.linspace(0, -30, 21)
results = []

for o2_limit in o2_range:
    rxn = model.reactions.get_by_id("EX_cpd00007_e")
    rxn.lower_bound = o2_limit
    rxn.upper_bound = 0

    try:
        sol = pfba(model)
        flux = sol.fluxes["EX_CLPbio_c"] if sol.status == 'optimal' else 0
    except:
        flux = 0

    results.append({'O2_uptake': -o2_limit, 'Product_flux': flux})
    print(f"O2: {-o2_limit:5.1f} | Product: {flux:.6f}")

# 保存CSV
df = pd.DataFrame(results)
df.to_csv("oxygen_sensitivity.csv", index=False)

# 绘图
plt.figure(figsize=(8, 5))
plt.plot(df['O2_uptake'], df['Product_flux'], 'b-o', linewidth=2, markersize=6)
plt.fill_between(df['O2_uptake'], 0, df['Product_flux'], alpha=0.3)
plt.xlabel('Oxygen Uptake Rate (mmol/gDCW/h)', fontsize=12)
plt.ylabel('EX_CLPbio_c Flux (mmol/gDCW/h)', fontsize=12)
plt.title('Product Yield vs Oxygen Uptake Rate', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("oxygen_sensitivity.png", dpi=150)
plt.show()

# 多条件模拟生长

## for m9

In [1]:
import pandas as pd
from cobra.io import read_sbml_model
from cobra import Reaction
from cobra.flux_analysis import pfba
import copy

def reset_model(model_original):
    """重置模型"""
    return model_original.copy()

def add_custom_reaction(model, reaction_id, metabolites_dict, reversible=True):
    """添加自定义反应"""
    rxn = Reaction(reaction_id)
    rxn.add_metabolites(metabolites_dict)
    rxn.lower_bound = -1000 if reversible else 0
    rxn.upper_bound = 1000
    model.add_reactions([rxn])

def apply_condition(model, condition):
    """应用测试条件"""
    # 关闭反应
    for rxn_id in condition.get('knockout', []):
        if rxn_id in model.reactions:
            model.reactions.get_by_id(rxn_id).knock_out()

    # 设置exchange反应
    for rxn_id, bound in condition.get('exchanges', {}).items():
        if rxn_id in model.reactions:
            rxn = model.reactions.get_by_id(rxn_id)
            rxn.lower_bound = bound
            rxn.upper_bound = max(0, bound) if bound < 0 else 1000

    # 添加外源反应
    if 'add_reactions' in condition:
        for rxn_id, metabolites in condition['add_reactions'].items():
            add_custom_reaction(model, rxn_id, metabolites)

# 加载模型
model_original = read_sbml_model("iCZ871_MM.xml")

# 定义所有测试条件
conditions = [
    {'name': 'GUR=1.48', 'exp': 0.024, 'exchanges': {'EX_cpd00027_e': -1.48, 'EX_cpd00007_e': 0}},
    {'name': 'GUR=2.68, OUR=0.57', 'exp': 0.101, 'exchanges': {'EX_cpd00027_e': -2.68, 'EX_cpd00007_e': -0.57}},
    {'name': 'GUR=2.06, OUR=3.29', 'exp': 0.197, 'exchanges': {'EX_cpd00027_e': -2.06, 'EX_cpd00007_e': -3.29}},
    {'name': 'GUR=2.33, OUR=5.5', 'exp': 0.197, 'exchanges': {'EX_cpd00027_e': -2.33, 'EX_cpd00007_e': -5.5}},
    {'name': 'GUR=2.34, OUR=6.61', 'exp': 0.197, 'exchanges': {'EX_cpd00027_e': -2.34, 'EX_cpd00007_e': -6.61}},
    {'name': 'GUR=5.25, OUR=-', 'exp': 0.59, 'exchanges': {'EX_cpd00027_e': -5.25}},
    {'name': 'GUR=4.67, OUR=-', 'exp': 0.443, 'exchanges': {'EX_cpd00027_e': -4.67}},

    {'name': 'Fructose=4.42', 'exp': 0.48, 'exchanges': {'EX_cpd00082_e': -4.42, 'EX_cpd00027_e': 0}},
    {'name': 'Sucrose=2.90', 'exp': 0.61, 'exchanges': {'EX_cpd00076_e': -2.90, 'EX_cpd00027_e': 0}},
    {'name': 'Fru=3.10+Glc=2.12', 'exp': 0.53, 'exchanges': {'EX_cpd00082_e': -3.10, 'EX_cpd00027_e': -2.12}},
    {'name': 'Fru=2.45+Glc=2.62', 'exp': 0.58, 'exchanges': {'EX_cpd00082_e': -2.45, 'EX_cpd00027_e': -2.62}},
    {'name': 'Fru=2.26+Glc=3.55', 'exp': 0.59, 'exchanges': {'EX_cpd00082_e': -2.26, 'EX_cpd00027_e': -3.55}},

    {'name': 'Δpgi, GUR=4.18', 'exp': 0.376, 'knockout': ['rxn00558_c'], 'exchanges': {'EX_cpd00027_e': -4.18}},

    {'name': 'Acetate=18.14', 'exp': 0.28, 'exchanges': {'EX_cpd00029_e': -18.14, 'EX_cpd00027_e': 0}},
    {'name': 'Ace=9.07+Glc=2.42', 'exp': 0.36, 'exchanges': {'EX_cpd00029_e': -9.07, 'EX_cpd00027_e': -2.42}},
    {'name': 'Propionate=5.27', 'exp': 0.07, 'exchanges': {'EX_cpd00141_e': -5.27, 'EX_cpd00027_e': 0}},

    {'name': 'Glycerol=6.3', 'exp': 0.29, 'exchanges': {'EX_cpd00100_e': -6.3, 'EX_cpd00027_e': 0},
     'add_reactions': {'CUSTOM_GlyK': {model_original.metabolites.cpd00002_c: -1, model_original.metabolites.cpd00100_c: -1,
                                         model_original.metabolites.cpd00008_c: 1, model_original.metabolites.cpd00067_c: 1,
                                         model_original.metabolites.cpd00080_c: 1}}},
    {'name': 'Gly=2.64+Glc=2.94', 'exp': 0.35, 'exchanges': {'EX_cpd00100_e': -2.64, 'EX_cpd00027_e': -2.94},
     'add_reactions': {'CUSTOM_GlyK': {model_original.metabolites.cpd00002_c: -1, model_original.metabolites.cpd00100_c: -1,
                                         model_original.metabolites.cpd00008_c: 1, model_original.metabolites.cpd00067_c: 1,
                                         model_original.metabolites.cpd00080_c: 1}}},

    {'name': 'XylA, Xylose=1.27', 'exp': 0.09, 'exchanges': {'EX_cpd00154_e': -1.27, 'EX_cpd00027_e': 0},
     'add_reactions': {'CUSTOM_XylA': {model_original.metabolites.cpd00154_c: -1, model_original.metabolites.cpd00259_c: 1}}},
    {'name': 'XylAB, Xylose=3.35', 'exp': 0.11, 'exchanges': {'EX_cpd00154_e': -3.35, 'EX_cpd00027_e': 0},
     'add_reactions': {'CUSTOM_XylAB': {model_original.metabolites.cpd00154_c: -1, model_original.metabolites.cpd00259_c: 1}}},
]

# 批量运行测试
results = []
for cond in conditions:
    model = reset_model(model_original)
    apply_condition(model, cond)
    model.objective = 'EX_biomass_c'
    try:
        solution = model.optimize()
        growth = solution.objective_value
    except:
        growth = 0

    results.append({
        'Condition': cond['name'],
        'EXP': cond['exp'],
        'SIM': growth,
        'Error(%)': round(abs(growth - cond['exp']) / cond['exp'] * 100, 2) if cond['exp'] > 0 else 0
    })

# 输出结果
df_results = pd.DataFrame(results)
print("\n=== 批量预测结果 ===")
print(df_results.to_string(index=False))

# 保存到Excel
df_results.to_excel("simulation_results_MM.xlsx", index=False)
print("\n结果已保存到 simulation_results.xlsx")

Set parameter Username
Set parameter LicenseID to value 2723056
Academic license - for non-commercial use only - expires 2026-10-16


No objective coefficients in model. Unclear what should be optimized


Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpn2p5xgz3.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15530 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpuhk7zzkr.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15530 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpmzkf2i5x.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15530 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpgoxhf68u.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15530 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpkoso3z_7.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15530 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmp0_7dokyo.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15530 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmp6yv0jxar.lp
Reading time = 0.01 seconds

## for CGXII

In [6]:
import pandas as pd
from cobra.io import read_sbml_model
from cobra import Reaction
from cobra.flux_analysis import pfba
import copy

def reset_model(model_original):
    """重置模型"""
    return model_original.copy()

def add_custom_reaction(model, reaction_id, metabolites_dict, reversible=True):
    """添加自定义反应"""
    rxn = Reaction(reaction_id)
    rxn.add_metabolites(metabolites_dict)
    rxn.lower_bound = -1000 if reversible else 0
    rxn.upper_bound = 1000
    model.add_reactions([rxn])

def apply_condition(model, condition):
    """应用测试条件"""
    # 关闭反应（当值为'-'时）
    for rxn_id in condition.get('knockout', []):
        if rxn_id in model.reactions:
            model.reactions.get_by_id(rxn_id).knock_out()

    # 设置exchange反应
    for rxn_id, bound in condition.get('exchanges', {}).items():
        if rxn_id in model.reactions:
            rxn = model.reactions.get_by_id(rxn_id)
            if bound == 'close':  # 当值为'-'时，关闭反应
                rxn.lower_bound = 0
                rxn.upper_bound = 0
            elif bound < 0:  # 负值表示摄入
                rxn.lower_bound = bound
                rxn.upper_bound = 0  # 只允许摄入，不允许分泌
            else:  # 正值表示分泌
                rxn.lower_bound = 0
                rxn.upper_bound = bound

    # 添加外源反应
    if 'add_reactions' in condition:
        for rxn_id, metabolites in condition['add_reactions'].items():
            add_custom_reaction(model, rxn_id, metabolites)

# 加载模型
model_original = read_sbml_model("iCZ871_CGXII.xml")

# 根据表格定义测试条件
conditions = [

    {'name': 'Table: GUR=2.33, OUR=5.36', 'exp': 0.2, 'exp_cer': 5.64,
     'exchanges': {'EX_cpd00027_e': -2.33, 'EX_cpd00007_e': -5.36, 'EX_cpd00197_e': 'close'}},

    {'name': 'Table: GUR=3.44, OUR=7.16', 'exp': 0.3, 'exp_cer': 7.3,
     'exchanges': {'EX_cpd00027_e': -3.44, 'EX_cpd00007_e': -7.16, 'EX_cpd00197_e': 'close'}},

    {'name': 'Table: GUR=4.27, OUR=8.15', 'exp': 0.4, 'exp_cer': 7.94,
     'exchanges': {'EX_cpd00027_e': -4.27, 'EX_cpd00007_e': -8.15, 'EX_cpd00197_e': 'close'}},

    {'name': 'Table: GUR=4.086, PUR=0.013', 'exp': 0.41, 'exp_cer': 8.4191,
     'exchanges': {'EX_cpd00027_e': -4.086,'EX_cpd00197_e': -0.013}},

    {'name': 'Table: GUR=4.8', 'exp': 0.42, 'exp_cer': None,
     'exchanges': {'EX_cpd00027_e': -4.8,'EX_cpd00197_e': 'close'}},

    {'name': 'Table: GUR=5.1', 'exp': 0.46, 'exp_cer': 11.42,
     'exchanges': {'EX_cpd00027_e': -5.1,'EX_cpd00197_e': 'close'}},

    {'name': 'Table: GUR=1.78', 'exp': 0.15, 'exp_cer': 5.02,
     'exchanges': {'EX_cpd00027_e': -1.78,'EX_cpd00197_e': 'close'}},
]

# 批量运行测试
results = []
for cond in conditions:
    model = reset_model(model_original)
    apply_condition(model, cond)
    model.objective = 'EX_biomass_c'

    try:
        solution = model.optimize()
        growth = solution.objective_value

        # 获取二氧化碳输出通量（CER）
        if 'EX_cpd00011_e' in model.reactions:
            cer_flux = solution.fluxes['EX_cpd00011_e']
        else:
            cer_flux = None

        # 获取其他关键通量
        gur_flux = solution.fluxes.get('EX_cpd00027_e', 0) if 'EX_cpd00027_e' in model.reactions else 0
        our_flux = solution.fluxes.get('EX_cpd00007_e', 0) if 'EX_cpd00007_e' in model.reactions else 0
        pur_flux = solution.fluxes.get('EX_cpd00197_e', 0) if 'EX_cpd00197_e' in model.reactions else 0

    except Exception as e:
        growth = 0
        cer_flux = None
        gur_flux = 0
        our_flux = 0
        pur_flux = 0
        print(f"优化失败 - {cond['name']}: {str(e)}")

    # 计算误差
    growth_error = round(abs(growth - cond['exp']) / cond['exp'] * 100, 2) if cond['exp'] > 0 else 0

    # 如果有实验CER值，计算CER误差
    cer_error = None
    if cond.get('exp_cer') is not None and cer_flux is not None:
        cer_error = round(abs(cer_flux - cond['exp_cer']) / cond['exp_cer'] * 100, 2) if cond['exp_cer'] > 0 else 0

    results.append({
        'Condition': cond['name'],
        'EXP_Growth': cond['exp'],
        'SIM_Growth': growth,
        'Growth_Error(%)': growth_error,
        'EXP_CER': cond.get('exp_cer', '-'),
        'SIM_CER': cer_flux if cer_flux is not None else '-',
        'CER_Error(%)': cer_error if cer_error is not None else '-',
        'GUR_flux': round(gur_flux, 4),
        'OUR_flux': round(our_flux, 4),
        'PUR_flux': round(pur_flux, 4)
    })

# 输出结果
df_results = pd.DataFrame(results)
print("\n" + "="*100)
print("=== 批量预测结果 ===")
print("="*100)
print(df_results.to_string(index=False))
print("="*100)

# 分别统计表格条件和原始条件的平均误差
table_results = [r for r in results if r['Condition'].startswith('Table:')]
original_results = [r for r in results if r['Condition'].startswith('Original:')]

if table_results:
    avg_growth_error_table = sum([r['Growth_Error(%)'] for r in table_results]) / len(table_results)
    cer_errors = [r['CER_Error(%)'] for r in table_results if isinstance(r['CER_Error(%)'], (int, float))]
    avg_cer_error_table = sum(cer_errors) / len(cer_errors) if cer_errors else None

    print(f"\n表格条件平均生长误差: {avg_growth_error_table:.2f}%")
    if avg_cer_error_table is not None:
        print(f"表格条件平均CER误差: {avg_cer_error_table:.2f}%")

if original_results:
    avg_growth_error_original = sum([r['Growth_Error(%)'] for r in original_results]) / len(original_results)
    print(f"原始条件平均生长误差: {avg_growth_error_original:.2f}%")

# 保存到Excel
df_results.to_excel("simulation_results_CGXII.xlsx", index=False)
print(f"\n结果已保存到 simulation_results_CGXII.xlsx")

No objective coefficients in model. Unclear what should be optimized


Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmp287j51yp.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15528 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmp4fj17cwr.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15528 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmp8ftbgj4w.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15528 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpfznjsppz.lp
Reading time = 0.00 seconds
: 1660 rows, 3506 columns, 15528 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpdfv2gxgl.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15528 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpctz84awq.lp
Reading time = 0.01 seconds
: 1660 rows, 3506 columns, 15528 nonzeros
Read LP format model from file C:\Users\Asus\AppData\Local\Temp\tmpyesomeo1.lp
Reading time = 0.01 seconds